In [1]:
import numpy as np
import pandas as pd

In [2]:
csv = ['games.csv']

colum = ['id','rated','turns','victory_status','winner','increment_code','white_id','white_rating','black_id',
        'black_rating','opening_eco', 'opening_name','opening_ply']

In [3]:
chess = pd.DataFrame(columns = colum)

for file in csv:
    load_df = pd.read_csv(f'data/{file}')
    
    load_df = load_df[colum]
    
    chess = chess.append(load_df,ignore_index=True)


In [4]:
#Change format from object to create format
chess["rated"] = chess["rated"].astype('category')
chess["opening_eco"] = chess["opening_eco"].astype('category')
chess["turns"] = pd.to_numeric(chess["turns"])
chess["white_rating"] = pd.to_numeric(chess["white_rating"])
chess["black_rating"] = pd.to_numeric(chess["black_rating"])
chess["opening_ply"] = pd.to_numeric(chess["opening_ply"])


In [5]:
#drop unrated matches
chess['rated'].value_counts()
chess = chess[chess['rated'] == True]

In [6]:
#Drop plus from increment code
chess["increment_code"]= chess["increment_code"].str.split("+", n = 1, expand = True)

#Change Increment code format!
chess["increment_code"] = pd.to_numeric(chess["increment_code"])

In [7]:
#create loop to create format column

def func(x):
    if 0 < x < 2:
        return 'Bullet'
    elif 2 < x < 9:
        return 'Blitz'
    elif 9 < x < 19:
        return 'Rapid'
    return 'Classical'

chess['format']= chess["increment_code"].apply(func)


In [8]:
#narrow down columns
rated = chess[['id','format','victory_status','opening_ply','opening_name','winner','turns','white_id','white_rating','black_id',
        'black_rating']]
rated = rated.rename(columns={'opening_ply':'book_moves'})


In [9]:
rated.to_csv("data/chess.csv", index = False)


In [14]:
white = rated[['id','format','white_id','white_rating','victory_status','winner','opening_name','book_moves','turns','black_rating']]
black = rated[['id','format','black_id','black_rating','victory_status','opening_name','book_moves','winner','turns','white_rating']]

white = white.rename(columns={'id':'game_id'})
white = white.rename(columns={'white_id':'player_id'})
white = white.rename(columns={'white_rating':'player_rating'})


black = black.rename(columns={'id':'game_id'})
black = black.rename(columns={'black_id':'player_id'})
black = black.rename(columns={'black_rating':'player_rating'})


In [15]:
# create a list of our conditions for white
white_result = [
    (white['winner'] == 'white'),
    (white['winner'] == 'draw'),
    (white['winner'] == 'black')]

# create a list of the values we want to assign for each condition
values = ['Won', 'Draw', 'Lost']

# create a new column and use np.select to assign values to it using our lists as arguments
white['result'] = np.select(white_result, values)

#create a new column with difference of white rating vs black rating
white['rating_difference'] = white['player_rating'] - white['black_rating']

#drop now unneccessary columns
white = white.drop(['victory_status', 'black_rating','winner'], axis=1)

# display updated DataFrame
white.head()

,game_id,format,player_id,player_rating,opening_name,book_moves,turns,result,rating_difference
1,l1NXvwaE,Blitz,a-00,1322,Nimzowitsch Defense: Kennedy Variation,4,16,Lost,61
2,mIICvQHh,Blitz,ischia,1496,King's Pawn Game: Leonardis Variation,3,61,Won,-4
3,kWKvrqYL,Classical,daniamurashov,1439,Queen's Pawn Game: Zukertort Variation,3,61,Won,-15
4,9tXo1AUZ,Classical,nik221107,1523,Philidor Defense,5,95,Won,54
6,qwU9rasv,Rapid,capa_jr,1520,Blackmar-Diemer Gambit: Pietrowsky Defense,10,33,Won,97


In [16]:
# create a list of our outcomes from winner for black wins draw or loss
black_result = [
    (black['winner'] == 'black'),
    (black['winner'] == 'draw'),
    (black['winner'] == 'white')]

# create a list of the values we want to assign for each condition
values = ['Won', 'Draw', 'Lost']

# create a new column and use np.select to assign values to it using our lists as arguments
black['result'] = np.select(black_result, values)

#create a new column with difference of white rating vs black rating
black['rating_difference'] = black['player_rating'] - black['white_rating']

#drop now unneccessary columns
black = black.drop(['victory_status', 'white_rating','winner'], axis=1)

black.head()

,game_id,format,player_id,player_rating,opening_name,book_moves,turns,result,rating_difference
1,l1NXvwaE,Blitz,skinnerua,1261,Nimzowitsch Defense: Kennedy Variation,4,16,Won,-61
2,mIICvQHh,Blitz,a-00,1500,King's Pawn Game: Leonardis Variation,3,61,Lost,4
3,kWKvrqYL,Classical,adivanov2009,1454,Queen's Pawn Game: Zukertort Variation,3,61,Lost,15
4,9tXo1AUZ,Classical,adivanov2009,1469,Philidor Defense,5,95,Lost,-54
6,qwU9rasv,Rapid,daniel_likes_chess,1423,Blackmar-Diemer Gambit: Pietrowsky Defense,10,33,Lost,-97


In [18]:
frames = [white, black]

players = pd.concat(frames)

players.head()

,game_id,format,player_id,player_rating,opening_name,book_moves,turns,result,rating_difference
1,l1NXvwaE,Blitz,a-00,1322,Nimzowitsch Defense: Kennedy Variation,4,16,Lost,61
2,mIICvQHh,Blitz,ischia,1496,King's Pawn Game: Leonardis Variation,3,61,Won,-4
3,kWKvrqYL,Classical,daniamurashov,1439,Queen's Pawn Game: Zukertort Variation,3,61,Won,-15
4,9tXo1AUZ,Classical,nik221107,1523,Philidor Defense,5,95,Won,54
6,qwU9rasv,Rapid,capa_jr,1520,Blackmar-Diemer Gambit: Pietrowsky Defense,10,33,Won,97


In [19]:
rated.to_csv("data/players.csv", index = False)